In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import gmean
import statsmodels.api as sm
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

## Selecting Stocks

In [2]:
# constants
P_THRESHOLD = 0.05
ALPHA_THRESHOLD = 0
BETA_THRESHOLD = 1
R_SQUARED_THRESHOLD = 0.05

# update the dates accordingly
END_DATE = datetime.today().strftime('%Y-%m-%d')
FIT_START_DATE = (
    datetime.strptime(END_DATE, '%Y-%m-%d') - timedelta(days=365*1)
).strftime('%Y-%m-%d')

TEST_START_DATE = (
    datetime.strptime(END_DATE, '%Y-%m-%d') - timedelta(days=365*5)
).strftime('%Y-%m-%d')

In [3]:
# getting tickers
market = '^GSPC'
risk_free = '^TNX' # 10 year treasury yield

# scrape all snp500 tickers
soup = BeautifulSoup(open('./data/scrape.html'), 'html.parser')
snp500_tickers = [a.text for a in soup.find_all('a', href=True)]

In [4]:
# get market data
# prepare market data
fit_df_f = yf.download(risk_free, start=FIT_START_DATE, end=END_DATE)
fit_df_m = yf.download(market, start=FIT_START_DATE, end=END_DATE)

# daily variable risk-free rate
fit_r_f = fit_df_f['Adj Close'].values / (100 * 252)
fit_r_f = np.delete(fit_r_f, 0) # drop first element to match length of market returns

# daily market returns
fit_r_m = fit_df_m['Adj Close'].pct_change().dropna().values

# daily excess returns
fit_ex_m = fit_r_m - fit_r_f

# do the same with multiple fit data
test_df_f = yf.download(risk_free, start=TEST_START_DATE, end=END_DATE)
test_df_m = yf.download(market, start=TEST_START_DATE, end=END_DATE)

test_r_f = test_df_f['Adj Close'].values / (100 * 252)
test_r_f = np.delete(test_r_f, 0) 

test_r_m = test_df_m['Adj Close'].pct_change().dropna().values

test_ex_m = test_r_m - test_r_f

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
def fit_regression(x: np.array, y: np.array, print_summary=False) -> dict:
    
    X = sm.add_constant(x)
    model = sm.OLS(y, X)
    results = model.fit()
    
    if print_summary:
        print(results.summary())
    
    r_squared = results.rsquared.round(4)
    alpha = results.params[0].round(4)
    beta = results.params[1].round(4)
    p_alpha = results.pvalues[0].round(4)
    p_beta = results.pvalues[1].round(4)
    mse = results.mse_resid.round(4)
    
    return {
        'beta': beta, 
        'alpha': alpha, 
        'r_squared': r_squared, 
        'p_alpha': p_alpha, 
        'p_beta': p_beta, 
        'mse': mse
    }

In [6]:
def test_constant(x: np.array, y: np.array, print_summary=False) -> dict:
    df = pd.DataFrame(x, columns=['R_m'])
    
    # generate time varying data
    n = len(x)
    t = np.arange(1, n+1)
    i = (t - (n+1)/2)/(n - 1)
    
    # interaction terms
    df['alpha_t'] = i
    df['beta_t'] = df['R_m'] * df['alpha_t']
    
    # fit multiple regression
    X = sm.add_constant(df)
    model = sm.OLS(y, X)
    results = model.fit()
    
    if print_summary:
        print(results.summary())

    p_alpha_t = results.pvalues['alpha_t'].round(4)
    p_beta_t = results.pvalues['beta_t'].round(4)
    
    return {'p_alpha_t': p_alpha_t, 'p_beta_t': p_beta_t}

In [7]:
def select_stocks(tickers: list, ex_r_m: np.array, mu_f: np.array, test_ex_r_m: np.array, test_mu_f: np.array) -> dict:
    selected_stocks = {}
    
    for ticker in tickers:
        print(f'Processing {ticker} ...')
        
        # download ticker data
        df = yf.download(ticker, start=FIT_START_DATE, end=END_DATE)
        
        # if ticker data is not available
        if df.empty:
            continue
        
        r_i = df['Adj Close'].pct_change().dropna().values
        
        # if lengths of market and ticker data are not the same
        if len(r_i) != len(ex_r_m):
            continue
        
        ex_r_i = r_i - mu_f
                
        params = fit_regression(ex_r_m, ex_r_i)
        
        # move on to next ticker if thresholds not met
        if params['p_alpha'] > P_THRESHOLD or params['alpha'] < ALPHA_THRESHOLD:
            # the null hypothesis that alpha is 0 cannot be rejected
            continue
        
        if params['r_squared'] < R_SQUARED_THRESHOLD:
            continue
        
        # fit multiple regression to test for constant alpha
        test_df = yf.download(ticker, start=TEST_START_DATE, end=END_DATE)
        
        test_r_i = test_df['Adj Close'].pct_change().dropna().values
        
        # if ticker data is not same length as market data, skip
        if len(test_r_i) != len(test_ex_r_m):
            continue
        
        test_ex_r_i = test_r_i - test_mu_f
        
        p_vals = test_constant(test_ex_r_m, test_ex_r_i)
        
        # move on if alpha is not constant
        if p_vals['p_alpha_t'] < P_THRESHOLD:
            # the null hypothesis that alpha is constant is rejected
            continue
        
        # if the stock passes all tests, add it to the selected list
        selected_stocks[ticker] = {
            'beta': params['beta'],
            'alpha': params['alpha'],
            'mse': params['mse'],
            'r_squared': params['r_squared'],
            'constant_beta': p_vals['p_beta_t'] > P_THRESHOLD
        }
    
    return selected_stocks

In [8]:
selected_stocks = select_stocks(snp500_tickers, fit_ex_m, fit_r_f, test_ex_m, test_r_f)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Processing AAPL ...
Processing MSFT ...
Processing NVDA ...



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Processing GOOGL ...
Processing GOOG ...
Processing AMZN ...
Processing META ...


Processing BRK.B ...


...

Processing MKTX ...
Processing MHK ...
Processing PARA ...
Processing GL ...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Processing CZR ...
Processing BWA ...
Processing FMC ...



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Processing ETSY ...
Processing IVZ ...
Processing AAL ...


In [9]:
selected_stocks

{'TRGP': {'beta': 0.5436,
  'alpha': 0.0018,
  'mse': 0.0002,
  'r_squared': 0.0834,
  'constant_beta': False},
 'GDDY': {'beta': 0.9809,
  'alpha': 0.0019,
  'mse': 0.0002,
  'r_squared': 0.2023,
  'constant_beta': True},
 'NRG': {'beta': 0.8311,
  'alpha': 0.0023,
  'mse': 0.0003,
  'r_squared': 0.1056,
  'constant_beta': True}}

## Constructing Portfolio

Here I used 1 year of data. 

In [10]:
# market variance
var_m = np.var(fit_ex_m)
var_m = var_m * 252

# expected returns of market
mu_m = gmean(fit_r_m + 1) - 1
mu_m = (1 + mu_m)**252 - 1

# expected returns of risk-free asset
mu_f = gmean(fit_r_f + 1) - 1
mu_f = (1 + mu_f)**252 - 1

In [11]:
mu = np.array([])

# CAPM expected returns
for params in selected_stocks.values():
    mu_i = mu_f + params['beta'] * (mu_m - mu_f)
    mu = np.append(mu, mu_i)
    
# construct covariance matrix
cov = np.zeros((len(selected_stocks), len(selected_stocks)))

for i, params_i in enumerate(selected_stocks.values()):
    for j, params_j in enumerate(selected_stocks.values()):
        if i == j:
            cov[i, j] = params_i['beta']**2 * var_m + params_i['mse']*252
        else:
            cov[i, j] = params_i['beta'] * params_j['beta'] * var_m
            
# inverse covariance matrix
inv_cov = np.linalg.inv(cov)

In [12]:
# calculate weights for tangent portfolio
ONES = np.ones(len(selected_stocks))

w_bar = np.dot(inv_cov, (mu - mu_f))
w_t = w_bar / np.dot(ONES, w_bar)

mu_t = np.dot(w_t, mu)
std_t = np.sqrt(np.dot(w_t, np.dot(cov, w_t)))

In [13]:
print('Selected stocks: \n')
for i, (ticker, params) in enumerate(selected_stocks.items()):
    beta = params['beta']
    mu_i = (100*mu[i])
    sigma_i = np.sqrt(cov[i, i])*100
    
    print(f'{ticker} (beta:{beta:.2f})')
    print(f'Expected return: {mu_i:.2f}%')
    print(f'Standard deviation: {sigma_i:.2f}% \n')

print(f'Tangent Portfolio:')
print(f'Expected return: {100*mu_t:.2f}%')
print(f'Standard deviation: {100*std_t:.2f}% \n')
print('Weights:')

for t, w in zip(selected_stocks.keys(), w_t):
    print(f'{t}: {100*w:.2f}%')

Selected stocks: 

TRGP (beta:0.54)
Expected return: 13.73%
Standard deviation: 23.27% 

GDDY (beta:0.98)
Expected return: 21.21%
Standard deviation: 25.03% 

NRG (beta:0.83)
Expected return: 18.65%
Standard deviation: 29.05% 

Tangent Portfolio:
Expected return: 18.57%
Standard deviation: 16.95% 

Weights:
TRGP: 26.15%
GDDY: 47.19%
NRG: 26.66%


In [14]:
print(selected_stocks.keys())

dict_keys(['TRGP', 'GDDY', 'NRG'])
